Quelques viz pour le plan

In [ ]:
import plotly.express as px
import plotly.colors as colors
import plotly.graph_objects as go
from plotly.subplots import make_subplots

to_plot=[
    #'Population totale',
    'Immigrante',
    'Réfugiée',
    'Population résidente non permanente',
    'Continent Asie',
    'Continent Europe',
    'Continent Afrique',
    'Continent Amériques',
    'Continent Oceanie et autres lieux de naissance',
    'Parlant anglais à la maison',
    'Parlant français à la maison',
]

df = pd.read_csv('../assets/data/arrondissements.csv')
df = df.drop([0, 1]).reset_index(drop=True)

fig = make_subplots(rows=len(to_plot), cols=2, shared_xaxes=True, vertical_spacing=0.02, horizontal_spacing=0.1)
for i, col_name in enumerate(to_plot):
    fig.add_trace(go.Bar(
        x=df['Arrondissement'], 
        y=df[col_name], 
        name=col_name, 
        legendgroup=col_name,
        showlegend=True,
        marker_color=colors.qualitative.Plotly[i%len(colors.qualitative.Plotly)]), row=i+1, col=1)
    fig.add_trace(go.Bar(
        x=df['Arrondissement'], 
        y=df[col_name]/df['Population totale'], 
        legendgroup=col_name,
        showlegend=False,
        marker_color=colors.qualitative.Plotly[i%len(colors.qualitative.Plotly)]), row=i+1, col=2)
fig.update_layout(
    legend=dict(
        orientation='h',
        yanchor='bottom',
        y=1.02,
        xanchor='right',
        x=1),
    width=1000,
    height=800)
fig.show()

In [ ]:
import plotly.express as px
import plotly.colors as colors
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

to_plot=[
    #'Population totale',
    'Immigrante',
    'Réfugiée',
    'Population résidente non permanente',
    #'Continent Asie',
    #'Continent Europe',
    #'Continent Afrique',
    #'Continent Amériques',
    #
    # 'Continent Oceanie et autres lieux de naissance',
    #'Parlant anglais à la maison',
    #'Parlant français à la maison',
]

df = pd.read_csv('../assets/data/arrondissements.csv')
df = df.drop([0, 1]).reset_index(drop=True)
for col_name in to_plot:
    df[col_name] = df[col_name]/df['Population totale']
print(df.loc[:, 'Arrondissement': 'Population totale'])

fig = px.parallel_coordinates(df, color='Population totale', dimensions=to_plot)

fig.show()

In [ ]:
import plotly.express as px
import plotly.colors as colors
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

to_plot=[
    #'Population totale',
    'Continent Asie',
    'Continent Europe',
    'Continent Afrique',
    'Continent Amériques',
    'Continent Oceanie et autres lieux de naissance',
    #'Parlant anglais à la maison',
    #'Parlant français à la maison',
]

df = pd.read_csv('../assets/data/arrondissements.csv')

print(*enumerate(df.columns), sep='\n')

fig = go.Figure(go.Pie(
    labels=to_plot,
    values=df.loc[0, to_plot], 
    name='Population totale'))

cols = [k for k in range(134, 193) if not df.columns[k].startswith('Continent')]

fig.show()
fig = go.Figure(go.Bar(
    x=df.columns[cols],
    y=df.iloc[0, cols],))
fig.update_layout(width=800)
fig.show()

In [ ]:
import plotly.express as px
import plotly.colors as colors
import plotly.graph_objects as go
from plotly.subplots import make_subplots

to_plot=[
    #'Population totale',
    'Continent Asie',
    'Continent Europe',
    'Continent Afrique',
    'Continent Amériques',
    'Continent Oceanie et autres lieux de naissance',
    #'Parlant anglais à la maison',
    #'Parlant français à la maison',
]

df = pd.read_csv('../assets/data/arrondissements.csv')

idx_continents = [k for k in range(134, 193) if df.columns[k].startswith('Continent')]
idx_countries = [k for k in range(134, 193) if not df.columns[k].startswith('Continent')]
parents_countries = ['Continent Amériques' for i in range(135, 146)]\
    + ['Continent Europe' for i in range(147, 163)]\
    + ['Continent Afrique' for i in range(164, 174)]\
    + ['Continent Asie' for i in range(175, 192)]

print(df.columns[idx_continents])
print(len(idx_continents), len(idx_countries), len(parents_countries))
print(df.iloc[0, idx_continents])
print(sum(df.iloc[0, idx_americas]))
idx_americas = [k for k in range(135, 146)]
idx_europe = [k for k in range(147, 163)]
idx_africa = [k for k in range(164, 174)]
idx_asia = [k for k in range(175, 192)]
idx_oceania = [192]

fig = go.Figure(go.Sunburst(
    labels=df.columns[idx_continents].to_list()\
        + df.columns[idx_americas].to_list()\
        + df.columns[idx_europe].to_list()\
        + df.columns[idx_africa].to_list()\
        + df.columns[idx_asia].to_list()\
        + df.columns[idx_oceania].to_list(),
    parents=['']*len(idx_continents)\
        + ['Continent Amériques']*len(idx_americas)\
        + ['Continent Europe']*len(idx_europe)\
        + ['Continent Afrique']*len(idx_africa)\
        + ['Continent Asie']*len(idx_asia)\
        + ['Continent Oceanie et autres lieux de naissance']*len(idx_oceania),
    values=df.iloc[0, idx_continents+idx_americas+idx_europe+idx_africa+idx_asia+idx_oceania],
    
    ))
fig.show()